In [3]:
!pip install transformers datasets accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import wandb

# Loading mini Flan dataset

In [6]:
df = pd.read_parquet("train-00000-of-00001.parquet")

In [4]:
df = df.iloc[:, :2]
df

,inputs,targets
0,"The thyroid gland, or simply the thyroid, is a...",no
1,Hayes High School (Birmingham Alabama) - Carol...,Animal
2,Answer the following question: Read the articl...,D
3,Please answer this: what is the charge for ent...,"June 24, 2017"
4,Write down the solution for this math problem:...,-5
...,...,...
143995,Problem: Build a movie plot around this: What ...,"In an unidentified city a gangster, Eddie Blac..."
143996,input question: Generate a question that has t...,What does Fiona do right before Shrek disappears?
143997,input: Please answer the following: Question: ...,Castle of Chaythe
143998,Build a movie plot around this: What has happe...,Despairing over the loss of a 19-year-old Mari...


# prompt

In [5]:
print(df.iloc[450,0])

Archaeological evidence shows that Homo erectus lived in the region now known as Myanmar as early as 400,000 years ago. The first evidence of Homo sapiens is dated to about 11,000 BC, in a Stone Age culture called the Anyathian with discoveries of stone tools in central Myanmar. Evidence of neolithic age domestication of plants and animals and the use of polished stone tools dating to sometime between 10,000 and 6,000 BC has been discovered in the form of cave paintings near the city of Taunggyi.

Answer this question, if possible (if impossible, reply "unanswerable"): What form was the evidence of ancient cultures discovered in ?


# target

In [6]:
print(df.iloc[450,1])

discovered in the form of cave paintings


source:`https://huggingface.co/datasets/pszemraj/flan-subsets-mini`

some ref:`https://huggingface.co/BEE-spoke-data/tFINE-680m-e32-d16-gqa-flan`

#for the forward

In [27]:
model_name = "EleutherAI/pythia-160m"  # for testing purposes smaller
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = "<pad>"  # seems without custom pad token set to zero the convergence behaves weirdly
tokenizer.pad_token_id = 0     # setting pad_token_id to 0

1

In [7]:
dataset = Dataset.from_pandas(df)

`note that in the Mini Flan dataset, the instruction and question are already merged into one as mentioned in the paper pg 5 for fine tuning`

`Original Sequence: What is the capital of France? → Paris`

`Tokenized input_ids: [A', B', C', D', E', F', G', H']`

`Correct labels: [B', C', D', E', F', G', H', -100'] (shift left).`

In [29]:
#forward tokenization
def preprocess_forward(examples):
    combined_text = [f"{inp}\n{target}" for inp, target in zip(examples['inputs'], examples['targets'])]
    tokenized_output = tokenizer(combined_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    input_ids = tokenized_output["input_ids"]
    labels = input_ids.clone()
    labels[:, :-1] = input_ids[:, 1:]  # shifting tokens to the right
    labels[:, -1] = -100  # the last token (no next token to predict) ignored
    tokenized_output["labels"] = labels
    return tokenized_output

tokenized_forward = dataset.map(preprocess_forward, batched=True)

Map:   0%|          | 0/144000 [00:00<?, ? examples/s]

`The following configs were used by others`

`learning_rate: 8e-05`

`train_batch_size: 4`

`eval_batch_size: 2`

`seed: 17868`

`distributed_type: multi-GPU`

`num_devices: 2`

`gradient_accumulation_steps: 32`

`total_train_batch_size: 256`

`total_eval_batch_size: 4`

`optimizer: Use paged_ademamix_32bit and the args are: No additional optimizer arguments`

`lr_scheduler_type: constant_with_warmup`

`lr_scheduler_warmup_ratio: 0.05`

`num_epochs: 1.0`

In [30]:
print("\nForward Direction - Tokenized Examples:")
for i in range(min(1, len(tokenized_forward))):
    print(f"Example {i + 1}:")
    print(f"Input IDs: {tokenized_forward[i]['input_ids']}")
    print(f"Decoded Tokens: {tokenizer.decode(tokenized_forward[i]['input_ids'], skip_special_tokens=True)}")
    print("-" * 50)


Forward Direction - Tokenized Examples:
Example 1:
Input IDs: [510, 17298, 21147, 13, 390, 3365, 253, 17298, 13, 310, 271, 35527, 21147, 275, 253, 7623, 13, 11253, 273, 767, 43253, 4802, 407, 271, 310, 7801, 316, 15, 733, 310, 1119, 387, 253, 2914, 273, 253, 7623, 13, 2708, 253, 13187, 434, 19126, 15, 380, 17298, 21147, 4279, 265, 17298, 23884, 13, 534, 8558, 4833, 253, 10761, 2281, 285, 2601, 9066, 15, 380, 23884, 671, 452, 1142, 643, 2538, 1690, 1110, 327, 2440, 15, 380, 17298, 23884, 1195, 1970, 16715, 1406, 460, 313, 53, 10, 285, 10386, 41636, 460, 313, 53, 10, 403, 3562, 432, 39887, 285, 25367, 15, 380, 17298, 671, 11330, 253, 14689, 9039, 24786, 249, 13, 534, 7120, 247, 2554, 275, 11672, 25641, 15, 187, 187, 1276, 434, 253, 1682, 3662, 281, 436, 1953, 27, 310, 253, 17298, 629, 273, 253, 6946, 985, 32, 187, 2369, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 50277, 5

In [31]:
#split the dataset
tokenized_forward = tokenized_forward.train_test_split(test_size=0.1)  # 90% train, 10% test

In [32]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

# loading the model for fine tuning
model_forward = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")

# hyperparameter config
training_args = TrainingArguments(
    output_dir="./fine-tuned-pythia-forward",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.05,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs-forward",
    logging_steps=10,
    push_to_hub=False,
)

# initilize of train
trainer_forward = Trainer(
    model=model_forward,
    args=training_args,
    train_dataset=tokenized_forward["train"],
    eval_dataset=tokenized_forward["test"],
    tokenizer=tokenizer,
)

trainer_forward.train()

# saving the model
model_forward.save_pretrained("./fine-tuned-pythia-forward")
tokenizer.save_pretrained("./fine-tuned-pythia-forward")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-32-4b710e223671>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_forward = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [25]:
wandb.finish()

train/epoch,▁▂▄▅▇█
train/global_step,▁▂▄▅▇█
train/grad_norm,▂█▆▁▇▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▄▁▂▂▂
train/epoch,0.0037
train/global_step,60
train/grad_norm,11.64044
train/learning_rate,1e-05
train/loss,3.7569


# for the backward

`Original Sequence: What is the capital of France? → Paris`

`Reversed Sequence: France of capital the is What → Paris`

`Tokenized input_ids: [A', B', C', D', E', F', G', H']`

`[H', G', F', E', D', C', B', A']`

`Correct labels: [-100', A', B', C', D', E', F', G'] (shift right).`

`[G', F', E', D', C', B', A', -100']`

In [8]:
model_name = "afterless/reverse-pythia-160m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = "<pad>"  # seems without custom pad token set to zero the convergence behaves weirdly
tokenizer.pad_token_id = 0     # setting pad_token_id to 0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/146 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [17]:
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer for this model doesn't have padding token, need to see how the explicit custom padding works
def preprocess_reverse(examples):
    # revering the order of inputs and targets at WORD level not char level
    reversed_inputs = [" ".join(inp.split()[::-1]) for inp in examples['inputs']]
    reversed_targets = [" ".join(target.split()[::-1]) for target in examples['targets']]
    combined_text = [f"{inp}\n{target}" for inp, target in zip(reversed_targets,reversed_inputs)]
    # tokenize
    tokenized_output = tokenizer(combined_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt", padding_side="left")

    # shifting input_ids to create labels (shift left) and base the loss on the first token
    input_ids = tokenized_output["input_ids"]
    labels = input_ids.clone()
    labels[:, 1:] = input_ids[:, 1:]  # shift tokens to the right
    labels[:, -1] = -100
    tokenized_output["labels"] = labels

    return tokenized_output

In [18]:
tokenized_backward = dataset.map(preprocess_reverse, batched=True)

Map:   0%|          | 0/144000 [00:00<?, ? examples/s]

In [19]:
print("\nBackward Direction - Tokenized Examples:")
for i in range(min(1, len(tokenized_backward))):
    print(f"Example {i + 1}:")
    print(f"Input IDs: {tokenized_backward[i]['input_ids']}")
    print(f"Decoded Tokens: {tokenizer.decode(tokenized_backward[i]['input_ids'], skip_special_tokens=True)}")
    print("-" * 50)


Backward Direction - Tokenized Examples:
Example 1:
Input IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [20]:
tokenized_backward = tokenized_backward.train_test_split(test_size=0.1)

In [21]:
# loading the model for fine tuning
model_backward = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./fine-tuned-pythia-backward",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs-backward",
    logging_steps=10,
    push_to_hub=False,
)

# initilize of train
trainer_backward = Trainer(
    model=model_backward,
    args=training_args,
    train_dataset=tokenized_backward["train"],
    eval_dataset=tokenized_backward["test"],
    tokenizer=tokenizer,
)

#.forward
trainer_backward.train()

# saving the model
model_backward.save_pretrained("./fine-tuned-pythia-backward")
tokenizer.save_pretrained("./fine-tuned-pythia-backward")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-21-18e3cae71add>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_backward = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [40]:
wandb.finish()

train/epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
train/global_step,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
train/grad_norm,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,██▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
train/loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,0.01173
train/global_step,190
train/grad_norm,0.21821
train/learning_rate,1e-05
train/loss,0.005
